In [ ]:
import numpy as np
import pandas as pd
import email
import re
from bs4 import BeautifulSoup

In [ ]:
EMAIL_FILE = 'file_path'

In [ ]:
emails_df = pd.read_csv(EMAIL_FILE)
print(emails_df.shape)
emails_df.head()

In [ ]:
## Helper functions
def get_text_from_email(msg):
    '''To get the content from email objects'''
    parts = []
    for part in msg.walk():
        if part.get_content_type() == 'text/plain':
            parts.append( part.get_payload() )
    return ''.join(parts)

def split_email_addresses(line):
    '''To separate multiple email addresses'''
    if line:
        addrs = line.split(',')
        addrs = frozenset(map(lambda x: x.strip(), addrs))
    else:
        addrs = None
    return addrs

In [ ]:
messages = list(map(email.message_from_string, emails_df['message']))
keys = messages[0].keys()
for key in keys:
    emails_df[key] = [doc[key] for doc in messages]
emails_df['content'] = list(map(get_text_from_email, messages))
emails_df['From'] = emails_df['From'].map(split_email_addresses)
emails_df['To'] = emails_df['To'].map(split_email_addresses)
emails_df['user'] = emails_df['file'].map(lambda x:x.split('/')[0])
del messages

emails_df.head()

In [ ]:
print (emails_df["content"][517377])

In [ ]:
OUTFILE = open('/deep/u/sxie/enron/enron_preprocess.txt', 'w+')

headers = ['Date', 'From', 'Sent', 'To', 'X-To', 'Subject', 'cc', 'Cc', 'CC', 'Attachments',
           'X-Mailer', 'MIME-Version', 'Content-Type', 'Precedence', 'Importance', 'FROM', 'TO', 'DATE', 'SENT', 'mailto']
skiplinestarters = ["<!--", '========', '?', '________']
skiplinetokens = ['[IMAGE]', '.pcx', '.DOC', 'Enron@ENRON', 'Enron@Enron', '---', 'dbCaps97Data', 'LOG MESSAGES:', 'TRANS_TYPE:'
                  '.doc', '.PDF', '.DOCX', '.jpg', '.JPG', '(E-mail)', '*****', 'ERROR: FILE', '.xls', '.XLS', '.zip', '.ppt'
                 '.PPT', '.PPTX', '.pptx', '.txt', '.TXT', '.csv', '.CSV', '.dat', '.DAT', 'href']
htmltokens = ['HTML', 'HEAD', 'TITLE', 'A']

replacements = {
    '=09': ''
}
replacement_starters = {
    '> ': '',
    '>': ''
}

valedictions = ['Best', 'Sincerely', 'Regards', 'Kind regards', 'Thanks', 'Cheers']

eos_punc = ['.', '!', '?']
sent_accumulator = []
prev_line_was_val = False

for email_body in emails_df["content"]:
    if 'Forwarded by' in email_body:
        continue
    for line in email_body.splitlines():
        line = line.strip()
        
        if any(('<%s>' % htmltoken) in line for htmltoken in htmltokens):
            break
        if any(('</%s>' % htmltoken) in line for htmltoken in htmltokens):
            break
        
        if not line:
            continue
        if any(skiptoken in line for skiptoken in skiplinetokens):
            continue
        if any(("%s:" % header) in line for header in headers):
            continue
        if any(line.startswith(starter) for starter in skiplinestarters):
            continue
        if bool(BeautifulSoup(line, "html.parser").find()):
            continue
            
        #handle spreadsheet type trade data
        if "COMMODITY" in line and "COUNTRY" in line and "CATEGORY" in line:
            continue
        
        if line.count(">") > 4:
            continue
        
        if line.endswith("=20"):
            line=line[:-3]
        
        for key in replacements:
            line = line.replace(key, replacements[key])
        
        for key in replacement_starters:
            if line.startswith(key):
                line = line[len(key):]
            
        if not line:
            continue

        if line.endswith('='):
            OUTFILE.write(line[:-1])
        elif any(line.startswith(val) for val in valedictions):
            OUTFILE.write('%s\n' % line)
            prev_line_was_val = True
        elif prev_line_was_val:
            prev_line_was_val = False
            OUTFILE.write('%s\n' % line)
        elif len(line.split(" "))<=1: 
            OUTFILE.write('%s\n' % line)
        elif re.match(r"[0-9\-\(\)]", line) and len(line.split(" "))<=3: #phone number
            OUTFILE.write('%s\n' % line)
        else:
            if (any(line.endswith(punc) for punc in eos_punc) or re.search(r"(\d{5})", line)):
                if len(sent_accumulator)>0:
                    sent_accumulator.append(line)
                    accumulation = ' '.join(sent_accumulator)
                    OUTFILE.write('%s\n' % accumulation)
                    sent_accumulator=[]
                else:
                    OUTFILE.write('%s\n' % line)
            else:
                sent_accumulator.append(line)